In [1]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from keras.preprocessing.image import img_to_array
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
h = 64       # height
w  = 256     # width

In [3]:
clean_data = []
path1 = 'D:\\intern\\classified_data\\clean_data\\clean_data\\'
files = os.listdir(path1)
for i in tqdm(files):
    img = cv2.imread(path1+i,0)   # change 0 to 1 for images
    img = cv2.resize(img,(h,w))
    clean_data.append(img_to_array(img))

100%|████████████████████████████████████████████████████████████████████████████| 69071/69071 [19:25<00:00, 59.27it/s]


In [5]:
noisy_data = []
path2 = 'D:\\intern\\classified_data\\noisy_data\\15dB\\15dB\\'
files = os.listdir(path2)
for i in tqdm(files):
    img = cv2.imread(path2+i,0)   # change 0 to 1 for images
    img = cv2.resize(img,(h,w))
    noisy_data.append(img_to_array(img))

100%|████████████████████████████████████████████████████████████████████████████| 69075/69075 [35:11<00:00, 32.72it/s]


In [6]:
noisy_train = np.reshape(noisy_data, (len(noisy_data), h, w, 1))
noisy_train = noisy_train.astype('float32') / 255.

clean_train = np.reshape(clean_data, (len(clean_data), h, w, 1))
clean_train = clean_train.astype('float32') / 255.

MemoryError: Unable to allocate 4.22 GiB for an array with shape (69071, 64, 256, 1) and data type float32

In [ ]:
# Displaying images with noise
plt.figure(figsize=(10,2))
for i in range(1,4):
    ax = plt.subplot(1,4,i)
    plt.imshow(noisy_train[i].reshape(h,w), cmap='binary')
plt.show()

In [ ]:
# Displaying images with noise
plt.figure(figsize=(10,2))
for i in range(1,4):
    ax = plt.subplot(1,4,i)
    plt.imshow(clean_train[i].reshape(h,w), cmap='binary')
plt.show()

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(h,w,1)))
model.add(MaxPooling2D((2,2), padding='same'))
model.add(Conv2D(8, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D((2,2), padding='same'))
model.add(Conv2D(8, (3,3), activation='relu', padding='same'))

model.add(MaxPooling2D((2,2), padding='same'))

model.add(Conv2D(8, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(8, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(1, (3,3), activation='relu', padding='same'))

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

model.summary()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(noisy_train, cleantrain,
                                                   test_size = 0.20, random_state = 0)

In [ ]:
model.fit(x_train, y_train, epochs=10, batch_size=8, shuffle=True, verbose=1, 
         validation_split= 0.1)

In [ ]:
print(f'Test_Accuracy: {model.evaluate(np.array(np.array(x_test), np.array(y_test))[1]*100)}')

In [ ]:
model.save('denoising_autoencoder.model')

In [ ]:
no_noise_img = model.predict(x_test)

In [ ]:
plt.imshow(no_noise_img[i].reshape(h,w), cmap='gray')